In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from util.analyze import Analyze
import numpy as np
from pprint import pprint
from pandas import DataFrame


class Bout(object):
    """
    A single walking bout.
    
    Walking-speed-param is calculated over the entire walking bout

    Value of any leg-motion-param is
     averaged over all values
     averaged over all legs in the specified body segment 
    """

    def __init__(self, path):
        """
        
        :param path: path to a single walking bout video that is leg-tracked 
        """
        
        ana = Analyze(path=path)

        self.walking_speed = int(ana.getWalkingSpeed()[0])

        body_segment = ('L2', 'R2', )

        swing_amplitudes = []
        swing_durations = []
        stance_amplitudes = []
        stance_durations = []
        # Concurrency states
        cc_state_0 = []
        cc_state_1 = []
        cc_state_2 = []
        cc_state_3 = []
        for leg in body_segment:
            swing_amplitudes.extend(ana.getSwingAmplitude(leg))
            swing_durations.extend(ana.getSwingDuration(leg))
            stamps = ana.getStanceAmplitude(leg)
            stance_amplitudes.extend(stamps)
            stance_durations.extend(ana.getStanceDuration(leg))
            # Concurrency states
            cc_state_0.extend(ana.getConcurrency('S0'))
            cc_state_1.extend(ana.getConcurrency('S1'))
            cc_state_2.extend(ana.getConcurrency('S2'))
            cc_state_3.extend(ana.getConcurrency('S3'))
            
        self.swing_amplitude = np.mean(swing_amplitudes, dtype=np.int)
        self.swing_duration = np.mean(swing_durations, dtype=np.int)
        self.stance_amplitude = np.mean(stance_amplitudes, dtype=np.int)
        self.stance_duration = np.mean(stance_durations, dtype=np.int)
        # Concurrency states
        self.cc_state_0 = np.mean(cc_state_0, dtype=np.int)
        self.cc_state_1 = np.mean(cc_state_1, dtype=np.int)
        self.cc_state_2 = np.mean(cc_state_2, dtype=np.int)
        self.cc_state_3 = np.mean(cc_state_3, dtype=np.int)

    def df(self):
        return DataFrame.from_records([self.__dict__])

path = '/media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/' \
       'RNAi/Constitutive/PanMn/PlexinAi/top30/22'

bout = Bout(path)
# pprint(bout.__dict__)
display(bout.df())

,cc_state_0,cc_state_1,cc_state_2,cc_state_3,stance_amplitude,stance_duration,swing_amplitude,swing_duration,walking_speed
0,11,33,15,38,2081,68,2086,25,28


In [3]:
from os import listdir
from os.path import isdir
from os.path import join as pjoin


class Trial(object):
    """
    A Trial contains many Bouts of a single genotype
    """
    
    def __init__(self, genotype, root_path):
        print('Analysing: %s' % genotype)
        print('@ %s' % root_path)
        self.genotype = genotype
        paths = [pjoin(root_path, path)
                    for path in sorted(listdir(root_path))
                 if isdir(pjoin(root_path, path)) and 'unusable' not in path and 'ignore' not in path]
        
        df = DataFrame()
        k = 1
        for path in paths:
            bout = Bout(path)
            df = df.append(bout.df(), ignore_index=True)
            k += 1
        print('Analysed %i walking bouts' % len(df.index))
        display(df)
        self.df = df

    def save_as_csv(self, path):
        fp = pjoin(path, self.genotype+'.csv')
        self.df.to_csv(fp)
        return fp

trial = Trial(
    genotype='UAS-Dicer2;OK371 GAL4 X UAS-Plexin-A RNAi',
    root_path='/media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/'
              'RNAi/Constitutive/PanMn/PlexinAi/top30'
)
file_path = trial.save_as_csv(
    path='/home/pushkar/Documents/Workspace/freewalk/data'
)
print('>>', file_path)

Analysing: UAS-Dicer2;OK371 GAL4 X UAS-Plexin-A RNAi
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/PanMn/PlexinAi/top30


Analysed 30 walking bouts


,cc_state_0,cc_state_1,cc_state_2,cc_state_3,stance_amplitude,stance_duration,swing_amplitude,swing_duration,walking_speed
0,15,52,25,5,843,88,1434,38,8
1,17,46,29,5,1178,93,1364,25,10
2,7,49,25,18,1433,96,1409,25,12
3,11,48,12,27,1862,70,1875,22,24
4,10,52,26,10,1126,60,1288,25,13
5,11,38,17,32,1464,85,1521,27,14
6,11,38,30,21,1240,97,1461,36,10
7,10,44,22,21,1521,82,1469,26,15
8,8,35,24,31,1864,61,1739,28,20
9,4,45,36,13,1536,73,1587,23,19


>> /home/pushkar/Documents/Workspace/freewalk/data/UAS-Dicer2;OK371 GAL4 X UAS-Plexin-A RNAi.csv


In [4]:
output_path = '/home/pushkar/Documents/Workspace/freewalk/data/csv/'
df_table_of_trials = DataFrame.from_csv('/home/pushkar/Documents/Workspace/freewalk/data/TableOfTrials.csv')

for i, row in df_table_of_trials.iterrows():
    trial = Trial(
        genotype=row.genotype,
        root_path=row.root_path
    )
    file_path = trial.save_as_csv(output_path)
    print(file_path)
    print()

Analysing: 48183-GAL4 X UAS-Rdli
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/48183 GAL4/top30


Analysed 30 walking bouts


,cc_state_0,cc_state_1,cc_state_2,cc_state_3,stance_amplitude,stance_duration,swing_amplitude,swing_duration,walking_speed
0,1,50,42,5,1450,96,1571,33,14
1,25,59,13,1,698,60,1039,21,8
2,45,41,9,3,354,40,522,21,4
3,11,36,16,35,1308,54,1209,25,18
4,40,43,14,1,783,44,1164,21,7
5,40,46,12,0,634,55,792,25,3
6,42,45,11,0,102,24,97,17,2
7,22,49,17,10,308,31,419,19,4
8,36,36,17,8,865,58,545,19,6
9,42,36,15,5,313,40,581,34,3


/home/pushkar/Documents/Workspace/freewalk/data/csv/48183-GAL4 X UAS-Rdli.csv

Analysing: VGNReg3-GAL4 x UAS-Rdli
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/Subsets/Rdli/VGN Intron Region 3 GAL4/Consolidated_VGNReg3_Rdli


Analysed 44 walking bouts


...

,cc_state_0,cc_state_1,cc_state_2,cc_state_3,stance_amplitude,stance_duration,swing_amplitude,swing_duration,walking_speed
0,11,85,3,0,1774,130,1774,31,11
1,28,58,7,5,1302,115,1373,25,9
2,10,51,33,5,1016,88,1743,38,9
3,18,54,24,3,649,49,1500,33,10
4,20,67,12,0,662,59,867,25,5
5,3,71,24,0,1524,115,1381,36,10
6,23,63,12,0,461,53,934,28,6
7,39,53,6,0,762,58,1362,35,5
8,11,50,25,13,824,70,1086,26,10
9,40,51,5,2,440,30,398,20,4


/home/pushkar/Documents/Workspace/freewalk/data/csv/Stum-204.csv

Analysing: For GABABi UAS-Dicer2;OK371 GAL4 X UAS-mCD8GFP
@ /media/pushkar/ccc1f18b-7f26-4c0d-b510-a582d28d075e/pushkar/walking/data/Swetha/analysis/RNAi/Constitutive/CONTROLS/GABABi Controls/videos


Analysed 24 walking bouts


,cc_state_0,cc_state_1,cc_state_2,cc_state_3,stance_amplitude,stance_duration,swing_amplitude,swing_duration,walking_speed
0,9,40,33,16,1341,76,1436,28,15
1,10,72,15,0,1294,110,1301,22,10
2,7,45,26,21,1538,70,1573,28,17
3,38,22,15,23,1058,55,1491,26,14
4,8,32,44,14,1368,62,1300,32,17
5,30,34,29,5,543,37,645,23,9
6,34,27,25,13,828,61,969,25,8
7,17,21,25,35,1632,70,1632,31,20
8,12,30,32,24,1425,60,1336,30,19
9,3,45,29,21,1532,66,1089,21,17


/home/pushkar/Documents/Workspace/freewalk/data/csv/For GABABi UAS-Dicer2;OK371 GAL4 X UAS-mCD8GFP.csv

